In [3]:
import sys; sys.path.insert(0, '..') # So that we import the local copy of pyzx if you have installed from Github
import random
import pyzx as zx
import os
import pandas as pd
import numpy as np

In [4]:
rule_dict = {'Spider Fusion Rule':'Spider', 'Identity Removal Rule' : 'IDRemoval', 'Pivot Rule': 'PIVOT', 'Local Complementation Rule':'LComp',
             'Phase Gadget Rule': 'PhaseGadget', 'Pivot Gadget Rule':'PIVOTGadget', 'Hadamard Rule':'HRule','Pivot Boundary Rule': 'PIVOTBoundary' }

def getInfo(cir):
    info = []
    info.append(cir.stats_dict()['gates'])
    info.append(cir.stats_dict()['clifford'])
    info.append(cir.stats_dict()['cnot'])
    info.append(cir.stats_dict()['tcount'])
    info.append(cir.depth())
    return info

def stepByStep(qasm: str, benchmark: str):
    new_infos, old_infos, new_idx, old_idx = [], [], ['original'] , ['original']
    c = zx.Circuit.load("../../../decomposition/qasm/"+benchmark+"_benchmark/"+qasm).to_basic_gates()
    new_infos.append(getInfo(c))
    old_infos.append(getInfo(c))
    folder_name = qasm.split('.')[0]
    PATH = "../../../decomposition/step-by-step/"+benchmark+"_benchmark/"+folder_name+"/"
    new_dict, old_dict = {}, {}
    for file in os.listdir(PATH+"new/"):
        idx = int(file.split("_")[0])
        new_dict[idx] = file
    for file in os.listdir(PATH+"old/"):
        idx = int(file.split("_")[0])
        old_dict[idx] = file
    
    for i in range(len(new_dict)):
        rule = new_dict[i].split("_")[1].split('.')[0]
        new_idx.append(rule_dict[rule])
        try:
            g = zx.Graph.from_zx(PATH+"new/"+new_dict[i])
            g.normalize()
            cir = zx.extract_circuit(g)
            new_infos.append(getInfo(cir))
        except:
            new_infos.append([None]*5)
            pass

    new_df = pd.DataFrame(new_infos)
    new_df.index = new_idx
    new_df.to_csv(PATH+"new.tsv", sep = '\t')
    print(new_df)
    
    for i in range(len(old_dict)):
        rule = old_dict[i].split("_")[1].split('.')[0]
        old_idx.append(rule_dict[rule])
        try:
            g = zx.Graph.from_zx(PATH+"old/"+old_dict[i])
            g.normalize()
            cir = zx.extract_circuit(g)
            old_infos.append(getInfo(cir))
        except:
            old_infos.append([None]*5)
            pass
    
    
    old_df = pd.DataFrame(old_infos)
    old_df.index = old_idx
    old_df.to_csv(PATH+"old.tsv", sep = '\t')
    print(old_df)



In [6]:
PyZX_benchmark = ["mod5_4.qasm","vbe_adder_3.qasm","csla_mux_3.qasm","csum_mux_9.qasm",
                  "qcla_com_7.qasm","qcla_mod_7.qasm","qcla_adder_10.qasm","adder_8.qasm",
                  "rc_adder_6.qasm","mod_red_21.qasm","mod_mult_55.qasm","barenco_tof_3.qasm",
                  "tof_3.qasm","barenco_tof_4.qasm","tof_4.qasm","barenco_tof_5.qasm",
                  "tof_5.qasm","barenco_tof_10.qasm","tof_10.qasm","gf2^4_mult.qasm",
                  "gf2^5_mult.qasm","gf2^6_mult.qasm","gf2^7_mult.qasm","gf2^8_mult.qasm"]
# "3_17_13.qasm","4gt11_82.qasm","4gt11_84.qasm","4gt13_92.qasm",
#                    "4mod5-v0_19.qasm","4mod5-v0_20.qasm","4mod5-v1_22.qasm","4mod5-v1_24.qasm",
#                    "alu-v0_27.qasm","alu-v1_28.qasm","alu-v1_29.qasm",
#                    "alu-v2_33.qasm","alu-v3_34.qasm","alu-v3_35.qasm","alu-v4_37.qasm","ex-1_166.qasm",
#                    "ham3_102.qasm","miller_11.qasm","mod5d1_63.qasm","mod5mils_65.qasm","qft_4.qasm",
#                    "rd32-v0_66.qasm","rd32-v1_68.qasm","cm42a_207.qasm","cm82a_208.qasm",
#                    "cycle10_2_110.qasm","dc2_222.qasm"

SARBE_benchmark_small = ["pm1_249.qasm","qft_10.qasm","qft_n20.qasm",
                   "rd53_251.qasm","rd73_252.qasm","sqrt8_260.qasm",
                   "square_root_7.qasm","z4_268.qasm",]

SARBE_benchmark_medium = ["sqn_258.qasm","ham15_107.qasm","inc_237.qasm","rd84_253.qasm",]

SARBE_benchmark_big = ["urf2_277.qasm","root_255.qasm","mlp4_245.qasm","life_238.qasm","hwb8_113.qasm","dist_223.qasm","cm85a_209.qasm","adr4_197.qasm","9symml_195.qasm"]

SARBE_benchmark_big_big = ["urf1_278.qasm"] # "urf1_149.qasm" <-- it is impossible to extract in a few minutes



for qasm in SARBE_benchmark_medium:
    print(qasm)
    stepByStep(qasm, "SARBE")
    print()


dc2_222.qasm


In [85]:
c = zx.Circuit.load("../../../decomposition/qasm/PyZX_benchmark/qcla_adder_10.qasm").to_basic_gates()
# f = open("../../../decomposition/qasm/PyZX_benchmark/qcla_adder_10.basic.qasm", 'a')
# f.write(c.to_qasm())
# f.close()

getInfo(c)
print("---------------")
PATH = "../../../decomposition/step-by-step/qcla_adder_10_basic/new/"
dict = {}
for file in os.listdir(PATH):
    idx = int(file.split("_")[0])
    rule = file.split("_")[1].split('.')[0]
    dict[idx] = file

for i in range(len(dict)):
    print(i, dict[i])
    try:
        g = zx.Graph.from_zx(PATH+dict[i])
        g.normalize()
        cir = zx.extract_circuit(g)
        getInfo(cir)
        print()
    except:
        print()
        pass

# for file in os.listdir(PATH):
#     try:
#         g = zx.Graph.from_zx(PATH+file)
#         g.normalize()
#         print(file)
#         cir = zx.extract_circuit(g)
#         printInfo(cir)
#         print()
#     except:
#         print()
#         pass

657 419 233 238 91
---------------
0 0_Spider Fusion Rule.zx

1 1_Identity Removal Rule.zx

2 2_Spider Fusion Rule.zx

3 3_Pivot Rule.zx

4 4_Local Complementation Rule.zx
951 739 17 212 146

5 5_Identity Removal Rule.zx
909 697 17 212 142

6 6_Spider Fusion Rule.zx
903 691 17 212 135

7 7_Pivot Rule.zx
899 687 19 212 135

8 8_Pivot Gadget Rule.zx
1030 818 109 212 293

9 9_Identity Removal Rule.zx

10 10_Spider Fusion Rule.zx
996 798 100 198 323

11 11_Pivot Rule.zx
982 784 121 198 365

12 12_Local Complementation Rule.zx
998 800 116 198 351

13 13_Identity Removal Rule.zx

14 14_Spider Fusion Rule.zx
971 775 114 196 277

15 15_Phase Gadget Rule.zx
934 772 89 162 344

16 16_Pivot Rule.zx
899 737 89 162 346

17 17_Local Complementation Rule.zx
859 697 89 162 321



In [72]:
c = zx.Circuit.load("../../../decomposition/qasm/PyZX_benchmark/mod_mult_55.qasm").to_basic_gates()
printInfo(c)
dict = {}
for g in c.gates:
    if type(g) == zx.gates.CNOT:
        if "CNOT" not in dict: dict["CNOT"] = 1
        else: dict["CNOT"] += 1
    elif type(g) == zx.gates.HAD:
        if "HAD" not in dict: dict["HAD"] = 1
        else: dict["HAD"] += 1
    elif type(g) == zx.gates.Tofolli:
        if "TOF" not in dict: dict["TOF"] = 1
        else: dict["TOF"] += 1
    elif type(g) == zx.gates.NOT:
        if "NOT" not in dict: dict["NOT"] = 1
        else: dict["NOT"] += 1
    elif type(g) == zx.gates.T:
        if "T" not in dict: dict["T"] = 1
        else: dict["T"] += 1
    else:
        print(type(g))
for i in dict:
    print(i, dict[i])

newGates = []
last = None
for i in range(0,len(c.gates)):
    if last == None:
        last = c.gates[i]
    else:
        if c.gates[i].name == 'HAD' and last.name == 'HAD' and c.gates[i].target == last.target:
            last = None
        else:
            newGates.append(last)
            last = c.gates[i]
if last != None:
    newGates.append(last)
print(len(c.gates),c.gates)
print(len(newGates),newGates, )
    

# g = c.to_graph()
# zx.simplify.full_reduce(g)
# cn = zx.extract_circuit(g)

# printInfo(cn)
# dict = {}
# for g in cn.gates:
#     if type(g) == zx.gates.CNOT:
#         if "CNOT" not in dict: dict["CNOT"] = 1
#         else: dict["CNOT"] += 1
#     elif type(g) == zx.gates.HAD:
#         if "HAD" not in dict: dict["HAD"] = 1
#         else: dict["HAD"] += 1
#     elif type(g) == zx.gates.Tofolli:
#         if "TOF" not in dict: dict["TOF"] = 1
#         else: dict["TOF"] += 1
#     elif type(g) == zx.gates.NOT:
#         if "NOT" not in dict: dict["NOT"] = 1
#         else: dict["NOT"] += 1
#     else:
#         print(type(g))
# for i in dict:
#     print(i, dict[i])




AttributeError: 'Circuit' object has no attribute 'stat'

In [63]:
c = zx.Circuit.load("../../../decomposition/qasm/PyZX_benchmark/csla_mux_3.qasm")
print(c.stats_dict())
c1 = zx.optimize.basic_optimization(c)
print(c1.stats())

{'name': 'csla_mux_3.qasm', 'qubits': 15, 'gates': 70, 'tcount': 70, 'clifford': 60, 'twoqubit': 20, 'cnot': 20, 'had': 40, 'measurement': 0, 'other': 10, 'depth': 0, 'depth_cz': 0}


TypeError: Unknown gate Tof(c1=1,c2=2,t=3)